<h2>Wrapper combining OpenAI, Melissa, Yelp</h2>

In [21]:
pip install xmltodict openai

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: xmltodict in c:\python3.11\lib\site-packages (0.13.0)




[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
import urllib.request, urllib.parse, os, json, xmltodict, asyncio, requests
from openai import AsyncOpenAI
from dotenv import load_dotenv
load_dotenv()


True

In [24]:
async def openai_wrapper(location, preferences = ""):
    # Get a list of 10 locations form OpenAI api
    openai_client = AsyncOpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    chat_completion = await openai_client.chat.completions.create(
        model="gpt-4-turbo-preview",
        response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": "You are a helpful travel assistant designed to output JSON containing a list of objects that have atrribute name and address."},
            {"role": "user", "content": f"I am traveling to {location}, and I want to visit attractions around this area that fit these criteria: {preferences}. Generate a list of 10 attraction objects and return each attraction name and entire address"}
        ],
    )
    return json.loads(chat_completion.choices[0].message.content)["attractions"]

In [25]:
async def yelp_wrapper(location, term = "attractions", radius = 20000, sort_by = "best_match", limit = 20):
    # Get a list of 20 locations form Yelp api
    params = {
        "location": location,
        "term": term,
        "radius": radius,
        "sort_by": sort_by,
        "limit": limit
    }
    yelp_url = f"https://api.yelp.com/v3/businesses/search?{urllib.parse.urlencode(params)}"
    yelp_header = {
        "accept": "application/json",
        "Authorization": f"Bearer {os.getenv('YELP_API_KEY')}"
    }
    response = await asyncio.to_thread(requests.get, yelp_url, headers = yelp_header)
    if response.status_code == 200:
        yelp_locations = response.json()
    else:
        response.raise_for_status()
    return yelp_locations
    

In [28]:
async def api_wrapper(location, preferences=""):
    # Run both openai_wrapper and yelp_wrapper concurrently
    openai_result, yelp_result = await asyncio.gather(
        openai_wrapper(location, preferences),
        yelp_wrapper(location, preferences)
    )
    return {"OpenAI": openai_result, "Yelp": yelp_result}

In [29]:
await api_wrapper("American High School, Fremont", "technology museum")

{'OpenAI': [{'name': 'Intel Museum',
   'address': '2200 Mission College Blvd, Santa Clara, CA 95054, United States'},
  {'name': 'Computer History Museum',
   'address': '1401 N Shoreline Blvd, Mountain View, CA 94043, United States'},
  {'name': 'The Tech Interactive',
   'address': '201 S Market St, San Jose, CA 95113, United States'},
  {'name': 'NASA Ames Visitor Center',
   'address': 'Space Exploration Center, Moffett Field, Mountain View, CA 94035, United States'},
  {'name': 'Silicon Valley Innovation Gallery',
   'address': '201 S Market St, San Jose, CA 95113, United States'},
  {'name': 'Google Android Lawn Statues',
   'address': '1981 Landings Dr, Mountain View, CA 94043, United States'},
  {'name': 'Apple Park Visitor Center',
   'address': '10600 N Tantau Ave, Cupertino, CA 95014, United States'},
  {'name': 'Silicon Valley Archives at Stanford',
   'address': '557 Escondido Mall, Stanford, CA 94305, United States'},
  {'name': 'Hiller Aviation Museum',
   'address': '6